**Homework Assignment 3 - Emilio Cabrera, Rushiil Deshmukh, Troy Richard**

In [79]:
import os
os.chdir('C:\\Users\\rushi\\Desktop\\Data Science\\HW3')
from pandas import Series, DataFrame
import pandas as pd
%pylab inline
import statsmodels.api as sm
from patsy import dmatrices

Populating the interactive namespace from numpy and matplotlib


**Question 1**

In [80]:
goldcolumns=['url','category']
labelcolumns=['turk','url','category']
goldata=pd.read_csv('gold.txt',sep='\t',names=goldcolumns) #Separated by a tab
labeldata=pd.read_csv('labels.txt',sep='\t',names=labelcolumns) #Separated by a tab

In [81]:
goldata[:10]

,url,category
0,http://0800-horoscope.com,G
1,http://18games.net,X
2,http://1pixelout.net,G
3,http://1porno.com/004/teens/index.html,X
4,http://1stpussy.com/005/16/index.html,X
5,http://3x-pix.com,X
6,http://4d.com,G
7,http://4neurons.com,G
8,http://4-panama-city-beach-condo.com/index.htm,G
9,http://4whw.8m.net/,X


In [82]:
labeldata[:10]

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
5,A2OVKTB7VNY8EW,http://000.cc,G
6,A31Z3E5SLATLML,http://000.cc,P
7,AAVCPZ8WTCFVK,http://000.cc,G
8,AHY98D5P05XIV,http://000.cc,G
9,APTFS1S88K5VC,http://000.cc,G


**Question 2**

In [83]:
mask=labeldata['url'].isin(goldata['url'])
labels_on_gold=labeldata[mask].drop_duplicates()
labels_unknown=labeldata[~mask].drop_duplicates()

The reason for dropping duplicates is: Thinking from the perspective of the organization which might use this crowdsourcing technique to train machines for url ratings, it would be practical to pick only the unique rating for a url given by a turk. This ensures a fair measure of the turk's accuracy, because multiple duplicate ratings will skew a turk's odds. 

In [84]:
labels_on_gold[:10]

,turk,url,category
245,A1253FXHCZ9CWM,http://0800-horoscope.com,G
246,A153PKAL7OAY36,http://0800-horoscope.com,G
247,A1FV9SAPL5C6KY,http://0800-horoscope.com,G
248,A1JTOT0DWM6QGL,http://0800-horoscope.com,G
249,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G
250,A21US576U8SCO4,http://0800-horoscope.com,G
251,A22C0PJUBFJTI0,http://0800-horoscope.com,G
252,A270HOAHEVIFW0,http://0800-horoscope.com,G
253,A2EG8FHODTEV63,http://0800-horoscope.com,R
254,A2LM400F4F0V0M,http://0800-horoscope.com,G


In [85]:
labels_unknown[:10]

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
5,A2OVKTB7VNY8EW,http://000.cc,G
6,A31Z3E5SLATLML,http://000.cc,P
7,AAVCPZ8WTCFVK,http://000.cc,G
8,AHY98D5P05XIV,http://000.cc,G
9,APTFS1S88K5VC,http://000.cc,G
10,A1PXXEOGQ76RNJ,http://0000.jp,G


**Question 3**

In [86]:
ratings = labels_on_gold.merge(goldata,left_on='url',right_on='url') #Merge to create a common ratings dataframe

counts = ratings.groupby('turk')['category_x'].count()
number = pd.DataFrame(counts) #Count of ratings for each turk

mask2 = (ratings['category_x'] == ratings['category_y'])
ratings['Correct'] = mask2
correct = ratings[['turk','Correct']].groupby('turk').mean() #Average rating correctness for each turk

rater_goodness=number.merge(correct,left_on='turk',right_on='turk')
rater_goodness.columns=['number of ratings','average correctness of ratings']

rater_goodness[:10]

,number of ratings,average correctness of ratings
turk,,
A112DVP1KG4QZU,1,1.000000
A1253FXHCZ9CWM,29,0.517241
A12CY1Q7XKJJDE,1,1.000000
A12RE8G66WTO8B,20,0.750000
A12Y1GTGIQDGRA,3,0.333333
A13CEW9JGDWGX1,1,1.000000
A13OE9GBRJ0S2U,4,0.750000
A14IQ4GLNWNPOJ,1,1.000000
A153PKAL7OAY36,148,0.722973


**Question 4**

In [87]:
rater_goodness['odds'] = rater_goodness['average correctness of ratings']/(1.001 - rater_goodness['average correctness of ratings'])
rater_goodness[:10]

,number of ratings,average correctness of ratings,odds
turk,,,
A112DVP1KG4QZU,1,1.000000,1000.000000
A1253FXHCZ9CWM,29,0.517241,1.069214
A12CY1Q7XKJJDE,1,1.000000,1000.000000
A12RE8G66WTO8B,20,0.750000,2.988048
A12Y1GTGIQDGRA,3,0.333333,0.499251
A13CEW9JGDWGX1,1,1.000000,1000.000000
A13OE9GBRJ0S2U,4,0.750000,2.988048
A14IQ4GLNWNPOJ,1,1.000000,1000.000000
A153PKAL7OAY36,148,0.722973,2.600369


**Question 5**

In [88]:
mask3 = rater_goodness['number of ratings'] >= 20
top_turks = rater_goodness[mask3].sort_values(by='average correctness of ratings',ascending=False)
top_turks[:10]

,number of ratings,average correctness of ratings,odds
turk,,,
A2U0R4X38GUKZE,20,0.950000,18.627451
A22C0PJUBFJTI0,36,0.916667,10.869565
A23YQUBXZPKILZ,24,0.875000,6.944444
ATVALOQVDCMZW,103,0.854369,5.826657
A1HIXWH4OXT8S4,40,0.825000,4.687500
A3220HG1O83HQ4,22,0.818182,4.475385
A32W20KGQXS0LL,25,0.800000,3.980100
A20PWAB7G3HDHU,20,0.800000,3.980100
AJSJVK40F5HM6,28,0.785714,3.649635


**Question 6**

In [89]:
rater_goodness['no'] = rater_goodness['number of ratings'] #Renaming column for matrix
rater_goodness['accuracy'] = rater_goodness['average correctness of ratings']
y, X=dmatrices('accuracy~no',data= rater_goodness,return_type='dataframe')
model=sm.OLS(y, X)
result=model.fit()
result.summary() #Summary statistics for the Regression plot

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               accuracy   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.5414
Date:                Tue, 03 Aug 2021   Prob (F-statistic):              0.463
Time:                        16:17:22   Log-Likelihood:                -94.561
No. Observations:                 269   AIC:                             193.1
Df Residuals:                     267   BIC:                             200.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6438      0.024     27.115      0.000       0.597       0.691
no             0.0007      0.001      0.736      0.463      -0.001       0.002
==============================================================================
Omnibus:                       29.163   Durbin-Watson:                   2.068
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.684
Skew:                          -0.738   Prob(JB):                     5.91e-07
Kurtosis:                       2.382   Cond. No.                         30.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [93]:
rater_goodness['number of ratings'].corr(rater_goodness['average correctness of ratings']) #Correlation co-efficients for the colu

0.044982640441412786

The number of ratings is statistically insignifcant i.e high p value = 0.463. The R squared value of 0.002 shows that the regression plot for accuracy vs number of ratings does not fit the data well. Furthermore, the correlation co-efficient between odds and number of ratings is 0.04498, which is a low value denoting that there is no significant relationship between the number of ratings per turk and their accuracy. 

**Question 7**

In [94]:
mask25 = rater_goodness['number of ratings'] > (rater_goodness['number of ratings'].quantile(0.75)) #Top 25%
best_turks = rater_goodness[mask25]
test_data = labels_unknown.merge(best_turks,left_on='turk',right_on='turk') #Merged with test data (unknown labels)

overall_odds = pd.DataFrame(test_data.groupby(['url','category'])['odds'].agg(product)) #Aggregate odds
overall_odds.columns=['overall odds']
overall_odds

overall odds
url                         category              
http://0-101.net            G             2.297602
http://000.cc               G             1.485456
http://0000.jp              G            14.339248
http://000relationships.com G             3.841962
                            P             1.851852
...                                            ...
http://zynga.com            G             3.666513
http://zz868.com            G             1.047852
                            P             1.851852
http://zzx.cc               G             1.417620
                            X             1.620746

[13983 rows x 1 columns]

**Question 8**

In [95]:
unstack_categories = overall_odds.unstack('category') #Searching odds for each category
top_category = pd.DataFrame(unstack_categories['overall odds'].T.idxmax())
top_odds = pd.DataFrame(unstack_categories['overall odds'].T.max())
result_75 = top_category.merge(top_odds, left_on = 'url', right_on = 'url')
result_75.columns = ['top category', ' top odds']
result_75

,top category,top odds
url,,
http://0-101.net,G,2.297602
http://000.cc,G,1.485456
http://0000.jp,G,14.339248
http://000relationships.com,G,3.841962
http://000vitamins.com,G,4.033641
...,...,...
http://zwinky.com,G,17.201875
http://zylom.com,G,1.849543
http://zynga.com,G,3.666513


**Question 9**

In [96]:
result_25 = rater_goodness['number of ratings'] > (rater_goodness['number of ratings'].quantile(.25)) #For top 75%
result_25_2 = rater_goodness[result_25]
labels_unknown_odds = labels_unknown.merge(result_25_2, left_on = 'turk', right_on = 'turk')
overall_odds_25 = DataFrame(labels_unknown_odds.groupby(['url', 'category'])['odds'].agg(prod))
overall_odds_25.columns = ['overall odds']

In [97]:
unstack_categories_25 = overall_odds_25.unstack('category') #Searching odds for each category
top_category_25 = DataFrame(unstack_categories_25['overall odds'].T.idxmax())
top_odds_25 = DataFrame(unstack_categories_25['overall odds'].T.max())
result_25 = top_category_25.merge(top_odds_25, left_on = 'url', right_on = 'url')
result_25.columns = ['top category', ' top odds']
result_25

,top category,top odds
url,,
http://0-101.net,G,2.297602e+00
http://000.cc,G,2.218192e+03
http://0000.jp,G,2.847963e+07
http://000relationships.com,P,1.851852e+00
http://000vitamins.com,G,5.343559e+00
...,...,...
http://zwinky.com,P,1.000000e+03
http://zylom.com,G,3.062113e+03
http://zynga.com,G,5.386158e+01


In [99]:
table_75_v_25 = rank_75.merge(result_25, left_on = 'url', right_on = 'url')
highest_error = pd.crosstab(table_75_v_25['top category_x'], table_75_v_25['top category_y'])
highest_error

top category_y,G,P,R,X
top category_x,,,,
G,8280,586,180,214
P,196,352,52,25
R,22,34,127,25
X,28,5,28,456


The total number of discrepancies for each category are as follows:  
G: 1110  
P: 429  
R: 186  
X: 463  

The discrepancy rates for each category are as follows:  
G: 10.58%  
P: 43.68%  
R: 38.94%  
X: 11.80%  

The number of discrepancies is highest for G (rated by top 25) rated as P by top 75. The discrepancy rate for P is the highest (43.68%), followed by rate for R (38.94%). 